#Image transformer

In [1]:
import os                 # For file and directory operations
import glob               # For flexible file pattern matching (e.g., *.jpg in folders)
from PIL import Image     # For opening, processing, and manipulating images

# Hugging Face Transformers pipeline utility for high-level model tasks (like classification or zero-shot)
from transformers import pipeline

# Gradio: A lightweight library to build web-based UIs for ML models
import gradio as gr


In [2]:
# === Initialize an Image Classification Pipeline Using Hugging Face Transformers ===

image_classifier = pipeline(
    "image-classification",           # Task type: image classification
    model="microsoft/resnet-50",      # Pretrained model: ResNet-50 trained on ImageNet-1k
    framework="pt"                    # Use PyTorch as the backend framework
)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [5]:
# === Step 2: Define the Prediction Function ===

def classify_image(image):
    # Ensure image is in RGB (some formats like .png may include alpha)
    image = image.convert("RGB")

    # Run image through the classifier pipeline
    outputs = image_classifier(image)

    # Convert list of outputs to a dictionary: {label: score}
    return {output["label"]: float(output["score"]) for output in outputs}


In [6]:
# === Step 3: Launch the Gradio Interface ===

gr.Interface(
    fn=classify_image,                      # The function to call on image input
    inputs=gr.Image(type="pil"),            # Accept a PIL image from upload or camera
    outputs=gr.Label(num_top_classes=2),    # Display top 2 class probabilities as labeled bars
    title="PCB Fault Detection",            # Web app title
    description="Upload a circuit board image to classify as Healthy or Faulty."  # App instructions
).launch()                                  # Start the web server and open in browser


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [12]:
# === Imports ===
import gradio as gr                    # For creating the web interface
from transformers import pipeline     # For loading pretrained transformer-compatible models
from pathlib import Path               # For handling filesystem paths
import os

# === Load Example Images ===

# Set the base dataset path
base_path = Path(r"D:\Cursor projects\Assignment2\Dataset_Circuitboard")

# Manually gather one sample from each class (assuming 'Healthy' and 'Faulty' folders exist)
healthy_folder = base_path / "Healthy"
faulty_folder = base_path / "Faulty"

# Collect one sample image path from each category
healthy = next((healthy_folder / f for f in os.listdir(healthy_folder) if f.endswith(('.jpg', '.png', '.jpeg'))))
faulty = next((faulty_folder / f for f in os.listdir(faulty_folder) if f.endswith(('.jpg', '.png', '.jpeg'))))

# === Load Image Classification Pipeline ===

pipe = pipeline(
    "image-classification",
    model="timm/resnet50.a1_in1k"  # Pretrained on ImageNet via TIMM
)


config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

d:\Cursor projects\Assignment2\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lille\.cache\huggingface\hub\models--timm--resnet50.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
def classify(image):
    # Ensure the input image is in RGB format
    # (some models don't handle grayscale or images with alpha channels well)
    image = image.convert("RGB")

    # Run the image through the Hugging Face image classification pipeline
    # This returns a list of predictions with labels and confidence scores
    predictions = pipe(image)

    # Return the top predicted label and its confidence score, formatted as a string
    return f"{predictions[0]['label']} ({predictions[0]['score']:.2f})"


In [13]:
# === Create and Launch a Gradio Interface ===

demo = gr.Interface(
    fn=classify,                     # The prediction function to call when an image is uploaded
    inputs=gr.Image(type="pil"),     # Input component: an image, returned as a PIL object
    outputs="text",                  # Output type: plain text showing predicted label and score
    title="Circuit Board Classifier",       # Title shown at the top of the web UI
    description="Upload a circuit board image to classify it.",  # Helpful description below the title
    examples=[[healthy], [faulty]]   # Preloaded image examples (optional, improves UX)
)

# === Start the App ===
demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
